# Week 09 Images 
## First a recap exercise
- https://api.chucknorris.io/
- Scraping all images from wikipedia (https://en.wikipedia.org/wiki/Chuck_Norris)

In [10]:
! pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\antho\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://api.chucknorris.io/jokes/random"
response = requests.get(url)
print(response.json()["value"])

urlWiki = "https://en.wikipedia.org/wiki/Chuck_Norris"
headers = {"User-Agent": "Anthony"}
responseWiki = requests.get(urlWiki, headers=headers)

soup = BeautifulSoup(responseWiki.text)
img_tags = soup.find_all("img")
#img_tags = soup.find_all("img", attrs="src") --> also a possibility, grabs your source!
for img in img_tags:
    if img["src"].endswith(".jpg"):
        parts = img["src"].split("/")

        with open("Chuck/" + parts[-1], "wb") as imagefile:
            response = requests.get("https:" + img["src"], headers=headers)
            imagefile.write(response.content)
            print("downloaded", parts[-1])            


## Don't forget to install Pillow package
## Computer images

https://pillow.readthedocs.io/

In [ ]:
! pip install Pillow

In [28]:
from PIL import ImageColor
print(ImageColor.getcolor('magenta', 'RGBA'))
print(ImageColor.getcolor('pink', 'RGBA'))
print(ImageColor.getcolor('chocolate', 'RGBA'))

(255, 0, 255, 255)
(255, 192, 203, 255)
(210, 105, 30, 255)


## Manipulating images

In [38]:
#PROPERTIES
from PIL import Image
img = Image.open("earth.jpg")
print(img)
print(img.height, img.width)
print(img.size)
print(img.filename)
print(img.format) #extension type

img.show()
img.save("earth.png") #changing extension type

#me opening and checking again
png = Image.open("earth.png")
png.show()
print(png.format_description)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x836 at 0x158B8888410>
836 1200
(1200, 836)
earth.jpg
JPEG
Portable network graphics


In [ ]:
#cropping
from PIL import Image

img = Image.open("earth.jpg")
img = img.crop((570, 276, 610, 316))
img.save("greenland.jpg")
img.show()


In [44]:
#creating images
from PIL import Image
img = Image.new('RGBA', (100,200), 'purple')
img.save("purple.png")
img.show()

img = Image.new('RGBA', (200,200), (0, 255, 0, 100))
img.save("green.png")
img.show()

In [53]:
from PIL import Image
img = Image.open("earth.jpg")
sea = img.crop((570, 316, 610, 356))
sea.save("sea.jpg")
greenland = Image.open("greenland.jpg")
sea.show()

copy = img.copy()
copy.paste(greenland, (0,0))
copy.paste(sea, (570, 276))
copy.show()

copy.save("greenlandGone.jpg")

In [60]:
#chessboard of greenland
from PIL import Image

canvas = Image.new('RGBA', (320,320), 'white')
greenland = Image.open("greenland.jpg")
sea = Image.open("sea.jpg")
copyGreenland = greenland.copy()
copySea = sea.copy()
canvas.paste(greenland, (0,0))
for i in range(0, 8):
    for j in range(0, 8):
        if (i + j) % 2 == 0:
            canvas.paste(copyGreenland, (i * greenland.width, j * greenland.height))
        else:
            canvas.paste(copySea, (i * sea.width, j * sea.height))

canvas.show()
canvas.save("chessboard.png")


In [65]:
#Transforming
from PIL import Image

img = Image.open("earth.jpg")
img.rotate(90).show() #left
img.rotate(-90).show() #right
img.rotate(180).show()

img.transpose(Image.FLIP_LEFT_RIGHT).show()
img.transpose(Image.FLIP_TOP_BOTTOM).show()

In [69]:
#looping over images pixel by pixel
from PIL import Image

img = Image.open("earth.jpg")
for x in range(0, img.width):
    for y in range(0, img.height):
        r,g,b = img.getpixel((x,y))
        inverted = (255-r, 255-g, 255-b)
        img.putpixel((x,y), inverted)

img.show()
img.save("inverted.png")

In [70]:
#removing blue from the image or something else
from PIL import Image

img = Image.open("earth.jpg")
for x in range(0, img.width):
    for y in range(0, img.height):
        r,g,b = img.getpixel((x,y))
        NOBLUE = (r, g, 0)
        img.putpixel((x,y), NOBLUE)

img.show()
img.save("noblue.png")

## object detection through opencv and numpy

In [76]:
! pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\antho\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
#regular contouring & manipulation
import cv2

img = cv2.imread("family.jpg")
print(img.shape) #an idea so you know it's still possible with cv2

grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("family_gray.jpg", grayscale)

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.imwrite("family_RGB.jpg", rgb)

##now the contouring part
#make a threshold for binary image
ret, thres = cv2.threshold(grayscale, 120, 255, 0)

contours, hierarchy = cv2.findContours(thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#drawing on the original
contour_img = img.copy()
cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
cv2.imwrite("family_contours.jpg", contour_img)


(1367, 2048, 3)


True

## OBJECT DETECTION
### classical approach

In [15]:
import cv2
img = cv2.imread("family.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors = 15)

for (x, y, z, a) in faces:
    cv2.rectangle(img, (x,y), (x + z, y + a), (255, 0, 0), 2)

cv2.imwrite("family_faces.jpg", img)

True

In [ ]:
#load the pretrained model
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
#detection on image and save

image_results = model("traffic.jpeg", save=True)
image_results = model("family.jpg", save=True)

video_results = model(source="crowd.mp4", save=True)

## Drawing on Images

In [ ]:
from PIL import Image, ImageDraw

img = Image.new('RGBA', (200, 200), 'white')
draw = ImageDraw.Draw(img)

draw.line([(0,0), (150, 50), (130, 100), (80, 150), (0, 0)], fill='black')
draw.rectangle((120,30, 160, 60), fill='red')

draw.text((20, 150), 'Hello World!', fill='purple')

img.save('drawing.png')